# Please UpVote of you like the notebook

### In this notebook we will see how to use HyperOpt which is a bayesian serach technique for finding the best loss

Importing libraries

In [12]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/hackerearth-effectiveness-of-std-drugs/dataset/test.csv
/kaggle/input/hackerearth-effectiveness-of-std-drugs/dataset/train.csv


Importing dataset

In [13]:
df= pd.read_csv("/kaggle/input/hackerearth-effectiveness-of-std-drugs/dataset/train.csv")

In [14]:
df.head(2)

,patient_id,name_of_drug,use_case_for_drug,review_by_patient,effectiveness_rating,drug_approved_by_UIC,number_of_times_prescribed,base_score
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27,8.022969
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192,7.858458


In [15]:
df.drop(["patient_id","name_of_drug","use_case_for_drug","review_by_patient","drug_approved_by_UIC"],axis=1,inplace=True)

In [16]:
min_rating = df.effectiveness_rating.min()
max_rating = df.effectiveness_rating.max()

def scale_rating(rating):
    rating -= min_rating
    rating = rating/(max_rating -1)
    rating *= 5
    rating = int(round(rating,0))
    
    if(int(rating) == 0 or int(rating)==1 or int(rating)==2):
        return 0
    else:
        return 1
    
df['new_eff_score'] = df.effectiveness_rating.apply(scale_rating)

#### Creating X and y

In [17]:
X = df.drop("base_score",axis=1)
y= df.base_score

#### Scaling

In [18]:
from sklearn.preprocessing import  MinMaxScaler
sc= MinMaxScaler()
X= sc.fit_transform(X)
y= y.values.reshape(-1,1)

### Hyperopt
Hyperopt is a python library for search spaces optimizing. Currently it offers two algorithms for minimizing the cost function,

- Random Search
- Tree of Parzen Estimators (TPE)

which is a Bayesian approach which makes use of P(x|y) instead of P(y|x), based on approximating two different distributions separated by a threshold instead of one in calculating the Expected Improvement

In [36]:
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from hyperopt import hp
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
# from sklearn.metrics import neg_mean_squared_error
from sklearn.metrics import mean_squared_error

In [60]:
def model(params):
    regr = xgb.XGBRegressor(**params)
    return cross_val_score(regr,X,y,scoring='neg_mean_squared_error').mean()

In [61]:
# XGB parameters
space = {
    'learning_rate':    hp.choice('learning_rate',    np.arange(0.15, 0.25, 0.01)),
    'n_estimators':     hp.choice('n_estimators', np.arange(1000,1800,10, dtype=int)),
    'max_depth':        hp.choice('max_depth',        np.arange(5, 15, 1, dtype=int)),
}

In [65]:
def objective(params):
    loss = model(params)
    return {'loss': -loss,'status' : STATUS_OK}

In [66]:
trials =Trials()

In [67]:
best = fmin(fn=objective,space=space,algo=tpe.suggest,max_evals=100,trials=trials)

100%|██████████| 100/100 [1:06:15<00:00, 39.75s/trial, best loss: 0.005281034326050954]


In [68]:
print(best)

{'learning_rate': 1, 'max_depth': 0, 'n_estimators': 63}


In [76]:
print("learning_rate = ",np.arange(0.15, 0.25, 0.01)[1])
print("max_dept = ",np.arange(5, 15, 1, dtype=int)[0])
print("n_estimators =",np.arange(1000,1800,10, dtype=int)[63])

learning_rate =  0.16
max_dept =  5
n_estimators = 1630


# Thank You!